In [144]:
workspace

'/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/01_mosaic'

In [224]:
[1,2]+[3,4]

[1, 2, 3, 4]

In [244]:
### Package Import
import sys
import os
base_dir = os.environ['GEMS_HOME']
project_path = os.path.join(base_dir, 'python-refactor')
sys.path.insert(0, project_path)
from Code.utils import matlab

import scipy.io as sio
import numpy as np
import glob
import time
import h5py 
import pygrib
import re
from osgeo import gdal
import tempfile
#import arcpy
#from arcpy import env


### Setting path
data_base_dir = os.path.join('/', 'media', 'sf_GEMS', 'Data')
raw_modis_path = os.path.join(data_base_dir, 'Raw', 'MODIS', 'MCD12Q1') 
path_mosaic = os.path.join(data_base_dir, 'Preprocessed_raw', 'MODIS', '01_mosaic') #workspace = os.path.join(work_path, '01_mosaic')
tmpdirname = tempfile.TemporaryDirectory(dir=base_dir)  # should call clean up to delete
#path_data="\\\\10.72.26.46\\irisnas6\\Data\\MODIS_tile\\"
#path_work=path_data+"02region\\EastAsia\\MCD12Q1\\"
#path_raw = path_data+"00raw\\MCD12Q1\\"

flist = glob.glob(os.path.join(raw_modis_path, "*.hdf"))
flist = [os.path.basename(f) for f in flist]
flist.sort()
nfile = len(flist)

for k in range(0,nfile,14):
    flist_temp = flist[k:k+14]
    yr = flist_temp[0][9:13]
    #print (flist_temp[0])
    #print (yr)

    for m in range(0,14):
        # Extract Subdataset
        fname = flist_temp[m]
        dst_dataset = os.path.join(tmpdirname.name, f"LC_{yr}_{m+1}.tif")
        #print (fname, dst_dataset)
        
        gdal_dataset = gdal.Open(os.path.join(raw_modis_path, fname))
        src_dataset = gdal_dataset.GetSubDatasets()[0][0]
        subprocess.call(["gdal_translate", dset, dst_dataset])
        
        if m==0:
            input_files = [dst_dataset]
        else:
            input_files.append(dst_dataset)
    
    # Mosaic
    matlab.check_make_dir(path_mosaic) # debugging
    fname = f"EA_MCD12Q1_mosaic_{yr}.tif"
    out_filename = os.path.join(path_mosaic, fname)
    pixel_type = 'Int16'
    cmd = ["gdal_merge.py", "-init", "255", "-ot", pixel_type, "-o", out_filename]
    cmd = cmd + input_files
    subprocess.call(cmd)
    
    tmpdirname.cleanup()
    print(yr)

2016


In [192]:
MyOut = subprocess.Popen(["gdal_merge.py", "-init", "255", "-ot", pixel_type, 
                 "-o", out_filename, " LC_2016_1.tif; LC_2016_2.tif"],
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT)
stdout, stderr = MyOut.communicate()
print(stdout)
print ("===========================")
print(stderr)

b'ERROR 4:  LC_2016_1.tif; LC_2016_2.tif: No such file or directory\nTraceback (most recent call last):\n  File "/usr/bin/gdal_merge.py", line 611, in <module>\n    sys.exit(main())\n  File "/usr/bin/gdal_merge.py", line 494, in main\n    ulx = file_infos[0].ulx\nIndexError: list index out of range\n'
None


In [68]:
from osgeo import gdal

In [122]:
fname = '/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf'
gdal_dataset = gdal.Open(fname)

In [124]:
gdal_dataset.GetSubDatasets()

[('HDF4_EOS:EOS_GRID:"/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf":MCD12Q1:LC_Type1',
  '[2400x2400] LC_Type1 MCD12Q1 (8-bit unsigned integer)'),
 ('HDF4_EOS:EOS_GRID:"/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf":MCD12Q1:LC_Type2',
  '[2400x2400] LC_Type2 MCD12Q1 (8-bit unsigned integer)'),
 ('HDF4_EOS:EOS_GRID:"/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf":MCD12Q1:LC_Type3',
  '[2400x2400] LC_Type3 MCD12Q1 (8-bit unsigned integer)'),
 ('HDF4_EOS:EOS_GRID:"/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf":MCD12Q1:LC_Type4',
  '[2400x2400] LC_Type4 MCD12Q1 (8-bit unsigned integer)'),
 ('HDF4_EOS:EOS_GRID:"/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf":MCD12Q1:LC_Type5',
  '[2400x2400] LC_Type5 MCD12Q1 (8-bit unsigned integer)'),
 ('HDF4_EOS:EOS_GRID:"/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2

In [73]:
lc_data = gdal.Open(gdal_dataset.GetSubDatasets()[0][0])

In [95]:
lc_data.ReadAsArray()

array([[ 3,  3,  8, ...,  9, 12, 14],
       [ 3,  3,  3, ...,  9, 14, 12],
       [ 3,  3,  3, ..., 14,  9,  9],
       ...,
       [16, 16, 16, ..., 16, 16, 16],
       [16, 16, 16, ..., 16, 16, 16],
       [16, 16, 16, ..., 16, 16, 16]], dtype=uint8)

In [ ]:
gdaltranslate  HDF4_EOS:EOS_GRID:"/path/to/file/MOD11A1.A2012193.h00v08.005.2012196013548.hdf":MODIS_Grid_Daily_1km_LST:LST_Day_1km out.tiff


In [96]:
dataset = gdal.Open('LC_2016_01.tif')

In [97]:
band = dataset.GetRasterBand(1)

In [98]:
band.ReadAsArray()

array([[ 3,  3,  8, ...,  9, 12, 14],
       [ 3,  3,  3, ...,  9, 14, 12],
       [ 3,  3,  3, ..., 14,  9,  9],
       ...,
       [16, 16, 16, ..., 16, 16, 16],
       [16, 16, 16, ..., 16, 16, 16],
       [16, 16, 16, ..., 16, 16, 16]], dtype=uint8)

In [106]:
cmd = 'gdal_translate -sds /media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf LC_2016.tif'

In [108]:
subprocess.call(["gdal_translate", "-sds", "/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf", "LC_2016.tif"])

0

In [110]:
subprocess.call(["rm", "LC_*", "-rf"])

0

In [136]:
dset

'HDF4_EOS:EOS_GRID:"/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf":MCD12Q1:LC_Type1'

In [103]:
import shlex

In [115]:
MyOut = subprocess.Popen(['gdalinfo', '-l', '.'], 
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT)
stdout, stderr = MyOut.communicate()
print(stdout)

b'total 26468\n-rw-r--r-- 1 sehyun sehyun       96 10\xec\x9b\x94 30 04:55 cache.hdf5\n-rw-r--r-- 1 sehyun sehyun     5390 11\xec\x9b\x94  2 21:14 GOCI_01_02.py\n-rwxr-xr-x 1 sehyun sehyun     4401 11\xec\x9b\x94  2 21:13 GOCI_01_extract_variables.py\n-rwxr-xr-x 1 sehyun sehyun     3079 11\xec\x9b\x94  2 21:13 GOCI_02_generate_missing_nan_file.py\n-rwxr-xr-x 1 sehyun sehyun     3112 11\xec\x9b\x94  2 21:29 GOCI_03_AOD_filter.py\n-rwxr-xr-x 1 sehyun sehyun     3914 11\xec\x9b\x94  3 14:54 GOCI_04_filtering.py\n-rwxr-xr-x 1 sehyun sehyun     4758 11\xec\x9b\x94  3 17:02 GOCI_05_BL_1km.py\n-rw-r--r-- 1 sehyun sehyun  1752184 10\xec\x9b\x94 29 21:48 GOCI_AOD_2016_001_01.mat\n-rw-r--r-- 1 sehyun sehyun     1312 10\xec\x9b\x94 29 21:50 GOCI_AOD_test\n-rw-r--r-- 1 sehyun sehyun     1312 10\xec\x9b\x94 29 21:50 GOCI_AOD_test.mat\n-rw-r--r-- 1 sehyun sehyun    65771 11\xec\x9b\x94  3 15:08 GOCI_recorrect.ipynb\n-rw-r--r-- 1 sehyun sehyun      399 11\xec\x9b\x94  2 15:08 GOCI_test.py\n-rwxr-xr-x

In [116]:
print(stderr)

None


In [40]:
import subprocess

In [46]:
src_dataset = os.path.join(raw_modis_path, a)
dst_dataset = b
subprocess.call(f'gdal_translate -of GTiff {src_dataset} {dst_dataset}')

FileNotFoundError: [Errno 2] No such file or directory: 'gdal_translate -of GTiff /media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf LC_2016_1.tif': 'gdal_translate -of GTiff /media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf LC_2016_1.tif'

In [57]:
subprocess.check_output(['gdal_translate', '-of', 'GTiff', 
                '/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf',
                'test.tif'])

CalledProcessError: Command '['gdal_translate', '-of', 'GTiff', '/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf', 'test.tif']' returned non-zero exit status 1.

In [80]:
gdal_translate -sds /media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf LC_2016.tif

'LC_2016_14.tif'

In [83]:
subprocess.check_call(['gdal_translate', '-sds', 
                '/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf',
                'LC_2016.tif'], shell=True)

CalledProcessError: Command '['gdal_translate', '-sds', '/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf', 'LC_2016.tif']' returned non-zero exit status 1.

In [64]:
result = subprocess.check_output('gdal_translate -of GTiff /media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf test.tif',stderr=subprocess.STDOUT, shell=True)

CalledProcessError: Command 'gdal_translate -of GTiff /media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/MCD12Q1.A2016001.h25v04.006.2018055072827.hdf test.tif' returned non-zero exit status 1.

# MODIS_02

In [335]:
### Package Import
import sys
import os
base_dir = os.environ['GEMS_HOME']
project_path = os.path.join(base_dir, 'python-refactor')
sys.path.insert(0, project_path)
from Code.utils import matlab

import scipy.io as sio
import numpy as np
import glob
import time
import h5py 
import pygrib
import re
from osgeo import gdal
import tempfile
#import arcpy
#from arcpy import env


### Setting path
data_base_dir = os.path.join('/', 'media', 'sf_GEMS', 'Data')
path_mosaic = os.path.join(data_base_dir, 'Preprocessed_raw', 'MODIS')
mask = os.path.join('/10.72.26.56', 'irisnas5', 'Data', 'mask', 'r_rec_N50W110S20E150.tif')
#tmpdirname = tempfile.TemporaryDirectory(dir=base_dir)  # should call clean up to delete
#path_modis="\\\\10.72.26.46\\irisnas6\\Data\\MODIS_tile\\02region\\EastAsia\\MCD12Q1\\"


flist = glob.glob(os.path.join(path_mosaic, '01mosaic', "*.tif"))
flist = [os.path.basename(f) for f in flist]
flist.sort()
nfile = len(flist)
print (flist)
#npath = len(path_modis+"\\01mosaic\\")

for fname in flist:
    src_dataset = os.path.join(path_mosaic, '01mosaic', fname) # a
    last_num = src_dataset[-8:] # b e.g. 2827.hdf
    print (src_dataset)
    
    matlab.check_make_dir(os.path.join(path_mosaic, '02prj_GCS_WGS84')) # debugging
    matlab.check_make_dir(os.path.join(path_mosaic, '03masked_N50W110S20E150')) # debugging
    
    dst_dataset02 = os.path.join(path_mosaic, '02prj_GCS_WGS84', f'GCS_EA_MCD12Q1_{last_num}') # c
    dst_dataset03 = os.path.join(path_mosaic, '03masked_N50W110S20E150', f'm_MODIS_LC_500m_{last_num}') # d

    #tempEnvironment0 = arcpy.env.snapRaster
    # Snap Raster—Sets a raster that is used to define the cell alignment of an output raster.
    # arcpy.env.snapRaster = mask
    
    # Process: Project Raster
    cell_size= ["5.11542231032757E-03", "5.11542231032757E-03"]
    out_coor_system = "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]"
    cmd = ["gdalwarp", "-r", "near", "-to", out_coor_system]
    cmd += (["-ts"] + cell_size)
    cmd += [src_dataset, dst_dataset02]
    #print (cmd)
    subprocess.call(cmd)
    
    """
    #arcpy.ProjectRaster_management(in_raster=a, 
    #                               out_raster=c,
    #                               out_coor_system="GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]",
    #                               resampling_type="NEAREST",
    #                               cell_size="5.11542231032757E-03 5.11542231032757E-03",
    #                               geographic_transform="",
    #                               Registration_Point="",
                                   in_coor_system="PROJCS['Unknown_datum_based_upon_the_custom_spheroid_Sinusoidal',GEOGCS['GCS_Unknown_datum_based_upon_the_custom_spheroid',DATUM['D_Not_specified_based_on_custom_spheroid',SPHEROID['Custom_spheroid',6371007.181,0.0]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Sinusoidal'],PARAMETER['false_easting',0.0],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',0.0],UNIT['Meter',1.0]]")
    """
    # Process: Extract by Mask
    cmd = ["gdaltindex", "temp_mask.shp", mask]
    print (cmd)
    subprocess.call(cmd)

    cmd = ["gdalwarp", "-cutline", "temp_mask.shp", 
           dst_dataset02, dst_dataset03]
    subprocess.call(cmd)

    cmd = ["rm", "temp_mask.shp"]
    subprocess.call(cmd)
    #arcpy.gp.ExtractByMask_sa(dst_dataset02, mask, dst_dataset03)
    #arcpy.env.snapRaster = tempEnvironment0
    
    print(last_num)

['EA_MCD12Q1_mosaic_2016.tif']
/media/sf_GEMS/Data/Preprocessed_raw/MODIS/01mosaic/EA_MCD12Q1_mosaic_2016.tif
['gdaltindex', 'temp_mask.shp', '/10.72.26.56/irisnas5/Data/mask/r_rec_N50W110S20E150.tif']
2016.tif


In [330]:
mask

'/10.72.26.56/irisnas5/Data/mask/r_rec_N50W110S20E150.tif'

In [332]:
cmd = ["gdaltindex", "temp_mask.shp", mask]
print (cmd)
subprocess.call(cmd)

cmd = ["gdalwarp", "-cutline", "temp_mask.shp", 
       dst_dataset02, dst_dataset03]
subprocess.call(cmd)

cmd = ["rm", "temp_mask.shp"]
subprocess.call(cmd)

['gdaltindex', 'mask.shp', '/10.72.26.56/irisnas5/Data/mask/r_rec_N50W110S20E150.tif']


0

In [328]:
mask

'/10.72.26.56/irisnas5/Data/mask/r_rec_N50W110S20E150.tif'

In [304]:
os.path.join(path_mosaic, '01_mosaic',)

'/media/sf_GEMS/Data/Preprocessed_raw/MODIS/01_mosaic'

In [305]:
tmp_fname = '/media/sf_GEMS/Data/Raw/MODIS/MCD12Q1/EA_MCD12Q1_mosaic_2016.tif'

In [306]:
src_dataset

'/media/sf_GEMS/Data/Preprocessed_raw/MODIS/01mosaic/EA_MCD12Q1_mosaic_2016.tif'

In [307]:
dst_dataset02

'/media/sf_GEMS/Data/Preprocessed_raw/MODIS/02prj_GCS_WGS84/GCS_EA_MCD12Q1_2016.tif'

In [322]:
cmd

['gdalwarp',
 '-r',
 'near',
 '-to',
 "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]",
 '/media/sf_GEMS/Data/Preprocessed_raw/MODIS/01mosaic/EA_MCD12Q1_mosaic_2016.tif',
 '/media/sf_GEMS/Data/Preprocessed_raw/MODIS/02prj_GCS_WGS84/GCS_EA_MCD12Q1_2016.tif']

In [325]:
cell_size= ["5.11542231032757E-03", "5.11542231032757E-03"]
out_coor_system = "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]"
cmd = ["gdalwarp", "-r", "near", "-to", out_coor_system]
cmd += (["-ts"] + cell_size)
cmd += [src_dataset, dst_dataset02]
print (cmd)
subprocess.call(cmd)

['gdalwarp', '-r', 'near', '-to', "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]", '-ts', '5.11542231032757E-03', '5.11542231032757E-03', '/media/sf_GEMS/Data/Preprocessed_raw/MODIS/01mosaic/EA_MCD12Q1_mosaic_2016.tif', '/media/sf_GEMS/Data/Preprocessed_raw/MODIS/02prj_GCS_WGS84/GCS_EA_MCD12Q1_2016.tif']


0

In [271]:
from osgeo import gdal,osr

In [273]:
# https://gis.stackexchange.com/questions/60371/gdal-python-how-do-i-get-coordinate-system-name-from-spatialreference
ds=gdal.Open(tmp_fname)
prj=ds.GetProjection()
print (prj)

PROJCS["unnamed",GEOGCS["Unknown datum based upon the custom spheroid",DATUM["Not_specified_based_on_custom_spheroid",SPHEROID["Custom spheroid",6371007.181,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]


In [274]:
prj

'PROJCS["unnamed",GEOGCS["Unknown datum based upon the custom spheroid",DATUM["Not_specified_based_on_custom_spheroid",SPHEROID["Custom spheroid",6371007.181,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'

In [272]:
srs=osr.SpatialReference(wkt=prj)
if srs.IsProjected:
    print (srs.GetAttrValue('projcs'))
print (srs.GetAttrValue('geogcs'))

unnamed
Unknown datum based upon the custom spheroid


In [ ]:

        subprocess.call(["gdal_translate", dset, dst_dataset])
        
        if m==0:
            input_files = [dst_dataset]
        else:
            input_files.append(dst_dataset)
    
    # Mosaic
    matlab.check_make_dir(path_mosaic) # debugging
    fname = f"EA_MCD12Q1_mosaic_{yr}.tif"
    out_filename = os.path.join(path_mosaic, fname)
    pixel_type = 'Int16'
    cmd = ["gdal_merge.py", "-init", "255", "-ot", pixel_type, "-o", out_filename]
    cmd = cmd + input_files
    subprocess.call(cmd)
    
    tmpdirname.cleanup()
    print(yr)